# COMP9318 Project
## Exploratory Data Analysis

In [11]:
import helper as h
import pandas as pd
import sklearn
import pandas2arff

Read in word data:

In [12]:
data_loc = 'asset/training_data.txt'
words = h.get_words(data_loc)

Check data frame

In [3]:
words.head()

,word,pronunciation,pn_list,destressed_pn_list,primary_stress_map,secondary_stress_map,primary_stress_idx,vowel_map,consonant_map,vector_map,type_tag,1st_letter_idx,phoneme_length,prefix,suffix
0,COED,K OW1 EH2 D,"[K, OW1, EH2, D]","[K, OW, EH, D]","[0, 1, 0, 0]","[0, 0, 1, 0]",2,"[0, 1, 1, 0]","[1, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...",NN,3,4,1,1
1,PURVIEW,P ER1 V Y UW2,"[P, ER1, V, Y, UW2]","[P, ER, V, Y, UW]","[0, 1, 0, 0, 0]","[0, 0, 0, 0, 1]",2,"[0, 1, 0, 0, 1]","[1, 0, 1, 1, 0]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, ...",NN,16,5,1,0
2,HEHIR,HH EH1 HH IH0 R,"[HH, EH1, HH, IH0, R]","[HH, EH, HH, IH, R]","[0, 1, 0, 0, 0]","[0, 0, 0, 0, 0]",2,"[0, 1, 0, 1, 0]","[1, 0, 1, 0, 1]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, ...",NN,8,5,0,0
3,MUSCLING,M AH1 S AH0 L IH0 NG,"[M, AH1, S, AH0, L, IH0, NG]","[M, AH, S, AH, L, IH, NG]","[0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0]",2,"[0, 1, 0, 1, 0, 1, 0]","[1, 0, 1, 0, 1, 0, 1]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",NN,13,7,0,1
4,NONPOISONOUS,N AA0 N P OY1 Z AH0 N AH0 S,"[N, AA0, N, P, OY1, Z, AH0, N, AH0, S]","[N, AA, N, P, OY, Z, AH, N, AH, S]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",5,"[0, 1, 0, 0, 1, 0, 1, 0, 1, 0]","[1, 0, 1, 1, 0, 1, 0, 1, 0, 1]","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",NN,14,10,1,1


In [5]:
words.columns

Index(['word', 'pronunciation', 'pn_list', 'destressed_pn_list',
       'primary_stress_map', 'secondary_stress_map', 'primary_stress_idx',
       'vowel_map', 'consonant_map', 'vector_map', 'type_tag',
       '1st_letter_idx', 'phoneme_length', 'prefix', 'suffix'],
      dtype='object')

Get pos_tags

In [10]:
columns = ['word','type_tag','1st_letter_idx','primary_stress_idx']
words_features  = words[columns]
words_features[words_features['1st_letter_idx']==11]


,word,type_tag,1st_letter_idx,primary_stress_idx
70,KURAS,NNP,11,2
73,KAPUR,NNP,11,2
74,KORBY,NNP,11,2
108,KEENER,NNP,11,2
126,KARCZEWSKI,NNP,11,4
187,KAWASAKI,NNP,11,6
196,KOZINSKI,NNP,11,4
208,KITTELL,NNP,11,2
215,KRISTINA,NNP,11,6
258,KARRICK,NNP,11,2


In [ ]:
%matplotlib inline

for pos_tag in pos_tags:
    temp_df = words_df[words_df['type_tag'] ==pos_tag]
    temp_df.plot(temp_df.primary_stress_idx,kind='scatter')